In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import hvplot.pandas

import warnings
warnings.filterwarnings('ignore')

In [6]:
# global variables 
trading_days_year = 252
total_days_year = 365
thirty_years_ago = (pd.Timestamp.today() - pd.Timedelta(days = total_days_year * 30)).date()
yesterday = (pd.Timestamp.today() - pd.Timedelta(days = 1)).date()
yesterday

datetime.date(2023, 4, 24)

In [37]:
# method for getting close prices on a list of stocks
def get_close(start, end, tickers = 'spy', single = True):
    
    if single:
        tickers = yf.Ticker(tickers)
        data = tickers.history(start=start, end=end)
        data = data['Close']
        data.dropna(inplace= True)
    else:
        tickers = yf.Tickers(tickers)
        data = tickers.history(start=start, end=end)
        data = data['Close']
        data.dropna(inplace= True)
    return data

In [39]:
bench = get_close(thirty_years_ago, yesterday, tickers = 'xlk xle xlf', single = False)
bench.hvplot()

[*********************100%***********************]  3 of 3 completed


:NdOverlay   [Variable]
   :Curve   [Date]   (value)